# Notebook 4 - Introducción a la Regressión Logística

Este notebook supone que han visto videos de presentación sobre la <b>Regresión Logística</b> para preparar la clase: <br>
- https://www.youtube.com/watch?v=gNhogKJ_q7U (11 min 52)
- https://www.youtube.com/watch?v=HFswrM68yPU (12 min 37)

La regresión logística es un modelo de clasificación que es fácil de implementar y que funciona muy bien en clases linealmente separables. Es uno de los algoritmos de clasificación más utilizados en la industria. El modelo de regresión logística un modelo lineal para la clasificación binaria que puede extenderse a la clasificación multiclase mediante la técnica OvR <a href="http://mlwiki.org/index.php/One-vs-All_Classification">("One-vs-Rest")</a>. 

## 1. Preparación del dataset 'Titanic'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sb.set_style('whitegrid')

Lo primero que vamos a hacer es leer en el dataset usando la función read_csv() de Pandas. Pondremos estos datos en un Pandas DataFrame, llamado "titanic", y nombraremos cada una de las columnas.

In [ ]:
url = 'https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv'
titanic = pd.read_csv(url)
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
titanic.head()

<b>DESCRIPCIONES VARIABLES</b>

Survived - Sobrevivencia (0 = No; 1 = Sí)<br>
PClass - Pasajero Clase (1 = 1º; 2 = 2º; 3 = 3º)<br>
Name - Nombre<br>
Sex - Genero<br>
Edad - Edad<br>
SibSp - Número de hermanos/cónyuges a bordo<br>
Parch - Número de padres/hijos a bordo<br>
Tickete - Número de billete<br>
Fare - Precio del tickete<br>
Cabin - Cabina<br>
Embarked - Puerto de Embarque (C = Cherbourg; Q = Queenstown; S = Southampton)<br>

<b>Comprobación de que la variable de destino es binaria</b>

Como estamos construyendo un modelo para predecir la sobrevivencia de los pasajeros del Titanic, nuestro objetivo va a ser la variable "Survived" del dataset del Titanic. Para asegurarnos de que es una variable binaria, usemos la función countplot() de Seaborn.

In [ ]:
sb.countplot(x='Survived',data=titanic, palette='hls')

Ok, entonces vemos que la variable Survived es binaria (0 - no sobrevivió / 1 - sobrevivió)

<b>Verificación de valores que faltan</b>

Es fácil comprobar si hay valores faltantes llamando al método isull(), y el método sum() fuera de él, para devolver un recuento de todos los valores Verdaderos que son devueltos por el método isnull().

In [ ]:
titanic.isnull().sum()

- ¿Cuántos registros hay en el dataset?

In [ ]:
titanic.info()

Ok, entonces sólo hay 891 filas en el marco de datos titánico. Cabin es casi todos los valores que faltan, así que podemos dejar esa variable completamente, pero ¿qué pasa con la edad? La edad parece ser un predictor relevante para la sobrevivencia, ¿verdad? Quisiéramos mantener las variables, pero le faltan 177 valores. Vamos a necesitar encontrar una manera de aproximar esos valores faltantes!

<b>Procesar los valores faltantes</b>

Así que sigamos adelante y saquemos del dataset todas las variables que no son relevantes para predecir la sobrevivencia. Al menos deberíamos mantener lo siguiente:

- Survived - Esta variable es obviamente relevante.<br>
- PClass - ¿Afecta la clase de un pasajero en el barco a su capacidad de sobreviviencia? <br>
- Sex - ¿Podría el género de un pasajero afectar su tasa de sobreviviencia? <br>
- Age - ¿Impacta la edad de una persona en su tasa de sobrevivencia? <br>
- SibSp - ¿El número de parientes en el barco (que son hermanos o cónyuge) afecta la sobrevivencia de una persona? <br>
- Parch - ¿El número de parientes en el barco (que son niños o padres) afecta la supervivencia de una persona? <br>
- Tarifa - ¿La tarifa que paga una persona afecta su sobrevivencia? Tal vez, conservémoslo. <br>
- Embarcado - ¿Importa el punto de embarque de una persona? Depende de cómo se llenó el barco... Vamos a conservarlo. <br>

¿Qué pasa con el nombre de una persona, el número de boleto y el número de identificación de pasajero? Son irrelevantes para predecir la supervivencia. Y como recordarán, la variable cabaña es casi todos los valores que faltan, así que podemos dejar todos estos.

In [ ]:
titanic_data = titanic.drop(['PassengerId','Name','Ticket','Cabin'], 1)
titanic_data.head()

Ahora tenemos el marco de datos reducido a sólo variables relevantes, pero ahora tenemos que ocuparnos de los valores que faltan en la variable de edad.

Veamos cómo se relaciona la edad del pasajero con su clase como pasajero en el barco.


In [ ]:
sb.boxplot(x='Pclass', y='Age', data=titanic_data, palette='hls')

In [ ]:
titanic_data.head()


Hablando a grandes rasgos, podríamos decir que cuanto más joven es un pasajero, más probable es que esté en tercera clase. Cuanto mayor sea el pasajero, mayor será la probabilidad de que esté en primera clase. Por lo tanto, existe una relación poco estrecha entre estas variables. Por lo tanto, escribamos una función que se aproxime a la edad de un pasajero, basada en su clase. Desde el punto de vista de la caja, parece que la edad media de los pasajeros de primera clase es de unos 37 años, la de los pasajeros de segunda clase es de 29 y la de los pasajeros de tercera clase es de 24 años.

Así que escribamos una función que encuentre cada valor nulo en la variable Age, y por cada nulo, verifique el valor de la Clase P y asigne un valor de edad de acuerdo a la edad promedio de los pasajeros en esa clase.

In [ ]:
def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

Cuando aplicamos la función y comprobamos de nuevo los valores nulos, vemos que no hay más valores nulos en la variable edad.

In [ ]:
titanic_data['Age'] = titanic_data[['Age', 'Pclass']].apply(age_approx, axis=1)
titanic_data.isnull().sum()

Hay 2 valores nulos en la variable embarcada. Podemos eliminar esos dos registros sin perder demasiada información importante de nuestro conjunto de datos, así que lo haremos.

In [ ]:
titanic_data.dropna(inplace=True)
titanic_data.isnull().sum()

<b>Conversión de variables categóricas en indicadores ficticios</b>

Lo siguiente que tenemos que hacer es reformatear nuestras variables para que funcionen con el modelo. Específicamente, necesitamos reformatear las variables Sexo y Embarcado en variables numéricas. 

In [ ]:
gender = pd.get_dummies(titanic_data['Sex'],drop_first=True)
gender.head()

In [ ]:
embark_location = pd.get_dummies(titanic_data['Embarked'],drop_first=True)
embark_location.head()

In [ ]:
titanic_data.drop(['Sex', 'Embarked'],axis=1,inplace=True)
titanic_data.head()

In [ ]:
titanic_dmy = pd.concat([titanic_data,gender,embark_location],axis=1)
titanic_dmy.head()

<b>¡Ahora tenemos un conjunto de datos con todas las variables en el formato correcto!</b>

PREGUNTAS:<br>
- ¿De qué sirve la fase de preparación del dataset?
- ¿En qué consiste la transformación de variables categóricas en variables 'ficticias'? ¿Por qué hacemos eso? 

## 2. Análisis de la corelación entre variables

In [ ]:
sb.heatmap(titanic_dmy.corr())

Fare y Pclass no son independientes el uno del otro, así que vamos a suprimir uno (Pclass por ejemplo).

In [ ]:
titanic_dmy.drop(['Pclass'],axis=1,inplace=True)
titanic_dmy.head()

In [ ]:
titanic_dmy.info()

Nos quedan 889 observaciones. Es suficiente para el aprendizaje.

- PREGUNTA: ¿Por qué sacamos las variables que parecen no ser independiente de otras variables antes la fase de aprendizaje?

## 3. Resolución del problema con Regresión Logística

Dividimos las columnas del dataset en dos partes: los <i>features</i> (X), y la variable que queremos predecir (y).

In [ ]:
X = titanic_dmy.ix[:,(1,2,3,4,5,6,7)].values
y = titanic_dmy.ix[:,0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

In [ ]:
#Aprendizaje
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

In [ ]:
print("coeficientes del modelo: "+str(LogReg.coef_))
print("intercept: "+str(LogReg.intercept_))

In [ ]:
#Predicciones
y_pred = LogReg.predict(X_test)
y_pred

In [ ]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

In [ ]:
print(classification_report(y_test, y_pred))

PREGUNTA:<br> 
- ¿Cómo pueden interpretar los resultados dados por la Matriz de Confusión y las métricas de Precision y Recall?
- ¿A qué corresponden los coeficientes del modelo?

## 4. Comparar los resultados de la Regresión Logística con kNN

PREGUNTA:
- ¿kNN es mejor que la Regresión Logística para resolver este problema? Demostrarlo con un análisis...

## 5. Utilizar un modelo de regresión linear para predecir la sobreviviencia

PREGUNTA:<br>
- ¿Por qué no podriamos tratar de utilizar una regresión linear para resolver el problema?

tratemos y veamos...

In [ ]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

In [ ]:
feature_cols = ['Age', 'SibSp', 'Parch', 'Fare', 'male', "Q", "S"]

# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

list(zip(feature_cols, linreg.coef_))

In [ ]:
# make predictions on the testing set
y_pred = linreg.predict(X_test)
y_pred

- PREGUNTA: ¿Qué pueden decir sobre las predicciones?

- PREGUNTA: Transformemos los valores de 'y_pred' en valores discretas (0 o 1) aplicando una regla básica tipo:
    - Si valor continua < 0.5 Entonces nueva valor = 0
    - Si valor continua >= 0.5 Entonces nueva valor = 1

- PREGUNTA: Analizar el rendimiento de la "regresión lineal" (ligeramente "hackeada") para resolver el problema de clasificación. Utilizar una matriz de confusión.